# **IMPORTANDO LIBRERIAS**

In [85]:
import sys
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy import create_engine, Column, Integer, String, VARCHAR,Boolean, TIMESTAMP,Float
from sqlalchemy.types import DateTime
from sqlalchemy.orm import sessionmaker


sys.path.append(os.path.abspath(os.path.join('..')))

In [86]:
load_dotenv()  # Carga las variables del archivo .env

config = {
    'host': os.getenv('HOST'),
    'user': os.getenv('USER'),
    'password': os.getenv('PASSWORD'),
    'database': os.getenv('DATABASE'),
    'port': os.getenv('PORT')
}

In [87]:
# Crear la URL de conexión para SQLAlchemy
db_url = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/caso_8"

# Crear el motor de SQLAlchemy
engine = create_engine(db_url, echo=True)



In [88]:
Base = declarative_base()


# **1. GENERACIÓN DE TABLAS**

In [89]:
class InterestMap(Base):

    __tablename__ = 'interest_map'
    __table_args__= {'extend_existing':True}

    id = Column(Integer, primary_key=True)
    interest_name = Column(VARCHAR(255))
    interest_summary = Column(VARCHAR(255))
    created_at = Column(TIMESTAMP)
    last_modified = Column(TIMESTAMP)

class InterestMetrics(Base):
    __tablename__ = 'interest_metrics'
    __table_args__ = {'extend_existing': True}

    id_ = Column(Integer, primary_key=True, autoincrement=True)
    _month = Column(VARCHAR(20))
    _year = Column(VARCHAR(20))
    month_year = Column(VARCHAR(20))
    interest_id = Column(VARCHAR(20))
    composition = Column(Float)
    index_value = Column(Float)
    ranking = Column(Integer)
    percentile_ranking = Column(Float)

Base.metadata.create_all(engine)


2025-05-06 13:33:11,228 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-05-06 13:33:11,228 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:11,229 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-05-06 13:33:11,230 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:11,232 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-05-06 13:33:11,232 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:11,234 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:11,234 INFO sqlalchemy.engine.Engine DESCRIBE `caso_8`.`interest_map`
2025-05-06 13:33:11,235 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:11,237 INFO sqlalchemy.engine.Engine DESCRIBE `caso_8`.`interest_metrics`
2025-05-06 13:33:11,238 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:11,239 INFO sqlalchemy.engine.Engine 
CREATE TABLE interest_map (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	interest_name VARCHAR(255), 
	interest_summary VARCHAR(255), 
	c

## **1.1 CARGANDO LOS DATOS**

In [90]:
from utils import funciones_utiles

interest_map_data = funciones_utiles.load_text_as_tuples('data/interest_map_data.txt')
interest_metrics_data = funciones_utiles.load_text_as_tuples('data/interest_metrics_data.txt')

In [91]:
# Creando la session
Session = sessionmaker(bind=engine)
session = Session()

### **1.1.1 Cargando datos `interest_map`**

In [92]:
# Obtener los ids existentes de InterestMap
existing_ids = {row.id for row in session.query(InterestMap.id).all()}

objects = [
    InterestMap(
        id=id,
        interest_name=interest_name,
        interest_summary=interest_summary,
        created_at=created_at,
        last_modified=last_modified
    )
    for id, interest_name, interest_summary, created_at, last_modified in interest_map_data
    if id not in existing_ids
]

if objects:
    session.bulk_save_objects(objects)
    session.commit()
else:
    print("No hay nuevos registros para insertar en InterestMap.")

2025-05-06 13:33:11,609 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:11,612 INFO sqlalchemy.engine.Engine SELECT interest_map.id AS interest_map_id 
FROM interest_map
2025-05-06 13:33:11,613 INFO sqlalchemy.engine.Engine [generated in 0.00108s] {}
2025-05-06 13:33:11,653 INFO sqlalchemy.engine.Engine INSERT INTO interest_map (id, interest_name, interest_summary, created_at, last_modified) VALUES (%(id)s, %(interest_name)s, %(interest_summary)s, %(created_at)s, %(last_modified)s)
2025-05-06 13:33:11,654 INFO sqlalchemy.engine.Engine [generated in 0.00369s] [{'id': '1', 'interest_name': 'Fitness Enthusiasts', 'interest_summary': 'Consumers using fitness tracking apps and websites.', 'created_at': '2016-05-26 14:57:59', 'last_modified': '2018-05-23 11:30:12'}, {'id': '2', 'interest_name': 'Gamers', 'interest_summary': 'Consumers researching game reviews and cheat codes.', 'created_at': '2016-05-26 14:57:59', 'last_modified': '2018-05-23 11:30:12'}, {'id': '3', 'interest

### **1.1.1 Cargando datos `interest_metrics`**

In [94]:
# Obtener los ids existentes de InterestMap
existing_ids = {row.id for row in session.query(InterestMetrics.id_).all()}

objects = [
    InterestMetrics(
        _month = None if _month == 'NULL' or _month == '' else _month,
        _year = None if _year == 'NULL' or _year == '' else _year,
        month_year = None if month_year == 'NULL' or month_year == '' else month_year,
        interest_id = None if interest_id == 'NULL' or interest_id == '' else interest_id,
        composition = composition,
        index_value = index_value,
        ranking = ranking,
        percentile_ranking = percentile_ranking
    )
    for _month, _year, month_year, interest_id, composition, index_value, ranking, percentile_ranking  in interest_metrics_data
    if id not in existing_ids
]

if objects:
    session.bulk_save_objects(objects)
    session.commit()
else:
    print("No hay nuevos registros para insertar en InterestMap.")

2025-05-06 13:33:34,041 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:34,043 INFO sqlalchemy.engine.Engine SELECT interest_metrics.id_ AS interest_metrics_id_ 
FROM interest_metrics
2025-05-06 13:33:34,044 INFO sqlalchemy.engine.Engine [generated in 0.00137s] {}
2025-05-06 13:33:34,571 INFO sqlalchemy.engine.Engine INSERT INTO interest_metrics (_month, _year, month_year, interest_id, composition, index_value, ranking, percentile_ranking) VALUES (%(_month)s, %(_year)s, %(month_year)s, %(interest_id)s, %(composition)s, %(index_value)s, %(ranking)s, %(percentile_ranking)s)
2025-05-06 13:33:34,573 INFO sqlalchemy.engine.Engine [generated in 0.04430s] [{'_month': '7', '_year': '2018', 'month_year': '07-2018', 'interest_id': '32486', 'composition': '11.89', 'index_value': '6.19', 'ranking': '1', 'percentile_ranking': '99.86'}, {'_month': '7', '_year': '2018', 'month_year': '07-2018', 'interest_id': '6106', 'composition': '9.93', 'index_value': '5.31', 'ranking': '2', 'perce

# Exploracion de datos y limpieza


In [95]:
query = '''

   SELECT 
    SUM(CASE WHEN id = 'None' OR id  OR id = '' THEN 1 ELSE 0 END) AS id_Nones,
    SUM(CASE WHEN interest_name = 'None' OR interest_name  OR interest_name = '' THEN 1 ELSE 0 END) AS interest_name_Nones,
    SUM(CASE WHEN interest_summary = 'None' OR interest_summary  OR interest_summary = '' THEN 1 ELSE 0 END) AS interest_summary_Nones,
    SUM(CASE WHEN created_at  THEN 1 ELSE 0 END) AS created_at_Nones,
    SUM(CASE WHEN last_modified  THEN 1 ELSE 0 END) AS last_modified_Nones
FROM interest_map;

'''

pd.read_sql_query(query, con=engine)

2025-05-06 13:33:38,484 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:38,486 INFO sqlalchemy.engine.Engine 

   SELECT 
    SUM(CASE WHEN id = 'None' OR id  OR id = '' THEN 1 ELSE 0 END) AS id_Nones,
    SUM(CASE WHEN interest_name = 'None' OR interest_name  OR interest_name = '' THEN 1 ELSE 0 END) AS interest_name_Nones,
    SUM(CASE WHEN interest_summary = 'None' OR interest_summary  OR interest_summary = '' THEN 1 ELSE 0 END) AS interest_summary_Nones,
    SUM(CASE WHEN created_at  THEN 1 ELSE 0 END) AS created_at_Nones,
    SUM(CASE WHEN last_modified  THEN 1 ELSE 0 END) AS last_modified_Nones
FROM interest_map;


2025-05-06 13:33:38,486 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:38,492 INFO sqlalchemy.engine.Engine ROLLBACK


,id_Nones,interest_name_Nones,interest_summary_Nones,created_at_Nones,last_modified_Nones
0,1209.0,0.0,20.0,1209.0,1209.0


In [96]:
query = '''

    SELECT 
        SUM(CASE WHEN _month = 'None' OR _month = '' THEN 1 ELSE 0 END) AS month_nulls,
        SUM(CASE WHEN _year = 'None' OR _year = '' THEN 1 ELSE 0 END) AS _year_nulls,
        SUM(CASE WHEN month_year = 'None'  OR month_year = '' THEN 1 ELSE 0 END) AS month_year_nulls,
        SUM(CASE WHEN interest_id = 'None'  OR interest_id = '' THEN 1 ELSE 0 END) AS interest_id_nulls,
        SUM(CASE WHEN index_value  = 'None'THEN 1 ELSE 0 END) AS index_value_nulls,
        SUM(CASE WHEN ranking  = 'None' THEN 1 ELSE 0 END) AS ranking_nulls,
        SUM(CASE WHEN percentile_ranking = 'None' THEN 1 ELSE 0 END) AS percentile_ranking_nulls

    FROM interest_metrics;

'''
pd.read_sql_query(query, con=engine)


2025-05-06 13:33:38,516 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:38,518 INFO sqlalchemy.engine.Engine 

    SELECT 
        SUM(CASE WHEN _month = 'None' OR _month = '' THEN 1 ELSE 0 END) AS month_nulls,
        SUM(CASE WHEN _year = 'None' OR _year = '' THEN 1 ELSE 0 END) AS _year_nulls,
        SUM(CASE WHEN month_year = 'None'  OR month_year = '' THEN 1 ELSE 0 END) AS month_year_nulls,
        SUM(CASE WHEN interest_id = 'None'  OR interest_id = '' THEN 1 ELSE 0 END) AS interest_id_nulls,
        SUM(CASE WHEN index_value  = 'None'THEN 1 ELSE 0 END) AS index_value_nulls,
        SUM(CASE WHEN ranking  = 'None' THEN 1 ELSE 0 END) AS ranking_nulls,
        SUM(CASE WHEN percentile_ranking = 'None' THEN 1 ELSE 0 END) AS percentile_ranking_nulls

    FROM interest_metrics;


2025-05-06 13:33:38,520 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:38,548 INFO sqlalchemy.engine.Engine ROLLBACK


,month_nulls,_year_nulls,month_year_nulls,interest_id_nulls,index_value_nulls,ranking_nulls,percentile_ranking_nulls
0,1194.0,1194.0,1194.0,1193.0,0.0,0.0,14.0


In [97]:
query = '''

    SELECT 
        DATE_FORMAT(STR_TO_DATE(month_year, "%%m-%%Y"), "01-%%m-%%Y") AS month_year_formatted
    FROM interest_metrics;

'''
pd.read_sql_query(query, con=engine)


2025-05-06 13:33:38,563 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:38,564 INFO sqlalchemy.engine.Engine 

    SELECT 
        DATE_FORMAT(STR_TO_DATE(month_year, "%%m-%%Y"), "01-%%m-%%Y") AS month_year_formatted
    FROM interest_metrics;


2025-05-06 13:33:38,564 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:38,647 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year_formatted
0,01-07-2018
1,01-07-2018
2,01-07-2018
3,01-07-2018
4,01-07-2018
...,...
14268,None
14269,None
14270,None
14271,None


In [98]:
with engine.begin() as connection:

    # Actualizar la columna para que tenga el formato correcto
    connection.execute(text('''
        UPDATE caso_8.interest_metrics
        SET month_year = DATE_FORMAT(
    STR_TO_DATE(
        NULLIF(NULLIF(NULLIF(month_year, 'NULL'), 'None'), ''), 
        '%m-%Y'
    ),
    '01-%m-%Y'
);
    '''))



2025-05-06 13:33:38,672 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:38,674 INFO sqlalchemy.engine.Engine 
        UPDATE caso_8.interest_metrics
        SET month_year = DATE_FORMAT(
    STR_TO_DATE(
        NULLIF(NULLIF(NULLIF(month_year, 'NULL'), 'None'), ''), 
        '%%m-%%Y'
    ),
    '01-%%m-%%Y'
);
    
2025-05-06 13:33:38,674 INFO sqlalchemy.engine.Engine [generated in 0.00074s] {}
2025-05-06 13:33:39,090 INFO sqlalchemy.engine.Engine COMMIT


In [99]:
query = '''

    SELECT 
        month_year
     FROM interest_metrics
     

'''
pd.read_sql_query(query, con=engine)

2025-05-06 13:33:39,109 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:39,110 INFO sqlalchemy.engine.Engine 

    SELECT 
        month_year
     FROM interest_metrics
     


2025-05-06 13:33:39,111 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:39,186 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year
0,01-07-2018
1,01-07-2018
2,01-07-2018
3,01-07-2018
4,01-07-2018
...,...
14268,None
14269,None
14270,None
14271,None


# **¿Cuál es el recuento de registros en fresh_segments.interest_metrics para cada valor de month_year ordenado en orden cronológico (del más antiguo al más reciente) con los valores nulos apareciendo primero?**


In [100]:
query = '''

    SELECT 
        month_year,
        COUNT(*) AS total_registros
    FROM interest_metrics
    GROUP BY 
        month_year
    
    ORDER BY total_registros ASC;


'''
pd.read_sql_query(query, con=engine)

2025-05-06 13:33:39,203 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:39,204 INFO sqlalchemy.engine.Engine 

    SELECT 
        month_year,
        COUNT(*) AS total_registros
    FROM interest_metrics
    GROUP BY 
        month_year
    
    ORDER BY total_registros ASC;



2025-05-06 13:33:39,204 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:39,223 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year,total_registros
0,01-07-2018,729
1,01-08-2018,767
2,01-09-2018,780
3,01-06-2019,824
4,01-10-2018,857
5,01-05-2019,857
6,01-07-2019,864
7,01-11-2018,928
8,01-01-2019,973
9,01-12-2018,995


# **¿Qué crees que deberíamos hacer con estos valores nulos en fresh_segments.interest_metrics?**


In [102]:
query = '''

    SELECT 
        COUNT(id_)
    FROM interest_metrics



'''
pd.read_sql_query(query, con=engine)

2025-05-06 13:33:48,988 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:33:48,989 INFO sqlalchemy.engine.Engine 

    SELECT 
        COUNT(id_)
    FROM interest_metrics




2025-05-06 13:33:48,990 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:33:48,993 INFO sqlalchemy.engine.Engine ROLLBACK


,COUNT(id_)
0,14273


In [104]:
query = '''
    WITH t1 AS (
    SELECT 
        month_year,
        COUNT(*) AS total_registros,
        ( SELECT COUNT(id_) FROM interest_metrics) AS total
    FROM interest_metrics
    GROUP BY 
        month_year
    
    ORDER BY total_registros ASC )

    SELECT
        month_year,
        total_registros*100.0 / total as pcge_total
    FROM t1

'''
pd.read_sql_query(query, con=engine)

2025-05-06 13:34:14,872 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:34:14,873 INFO sqlalchemy.engine.Engine 
    WITH t1 AS (
    SELECT 
        month_year,
        COUNT(*) AS total_registros,
        ( SELECT COUNT(id_) FROM interest_metrics) AS total
    FROM interest_metrics
    GROUP BY 
        month_year
    
    ORDER BY total_registros ASC )

    SELECT
        month_year,
        total_registros*100.0 / total as pcge_total
    FROM t1


2025-05-06 13:34:14,874 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:34:14,925 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year,pcge_total
0,01-07-2018,5.10755
1,01-08-2018,5.37378
2,01-09-2018,5.46486
3,01-06-2019,5.77314
4,01-10-2018,6.00434
5,01-05-2019,6.00434
6,01-07-2019,6.05339
7,01-11-2018,6.50179
8,01-01-2019,6.81707
9,01-12-2018,6.97120


Por lo tanto los valores nulos correspondel al 8.4 % lo que es poco y podriamos eliminar estos datos.

In [105]:
with engine.begin() as connection:

    # Actualizar la columna para que tenga el formato correcto
    connection.execute(text('''
        DELETE FROM interest_metrics
        WHERE interest_id IS NULL OR interest_id = 'None';
    '''))



2025-05-06 13:34:20,634 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:34:20,635 INFO sqlalchemy.engine.Engine 
        DELETE FROM interest_metrics
        WHERE interest_id IS NULL OR interest_id = 'None';
    
2025-05-06 13:34:20,636 INFO sqlalchemy.engine.Engine [generated in 0.00062s] {}
2025-05-06 13:34:20,664 INFO sqlalchemy.engine.Engine COMMIT


In [106]:
query = '''
    SELECT
        *
    FROM
        interest_metrics

'''
pd.read_sql_query(query, con=engine)

2025-05-06 13:34:20,686 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:34:20,687 INFO sqlalchemy.engine.Engine 
    SELECT
        *
    FROM
        interest_metrics


2025-05-06 13:34:20,688 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:34:20,926 INFO sqlalchemy.engine.Engine ROLLBACK


,id_,_month,_year,month_year,interest_id,composition,index_value,ranking,percentile_ranking
0,1,7,2018,01-07-2018,32486,11.89,6.19,1,99.86
1,2,7,2018,01-07-2018,6106,9.93,5.31,2,99.73
2,3,7,2018,01-07-2018,18923,10.85,5.29,3,99.59
3,4,7,2018,01-07-2018,6344,10.32,5.10,4,99.45
4,5,7,2018,01-07-2018,100,10.77,5.04,5,99.31
...,...,...,...,...,...,...,...,...,...
13075,13102,8,2019,01-08-2019,42011,1.77,0.84,1146,0.26
13076,13103,8,2019,01-08-2019,6065,2.13,0.80,1147,0.17
13077,13104,8,2019,01-08-2019,6050,1.90,0.79,1148,0.09
13078,13105,8,2019,01-08-2019,19599,1.66,0.77,1149,0.00


# **¿Cuántos valores de interest_id existen en la tabla fresh_segments.interest_metrics pero no en la tabla fresh_segments.interest_map? ¿Qué hay de la otra forma?**


In [ ]:
query = '''

    WITH full_outer_join_table AS (
   SELECT *
    FROM interest_metrics metrics
    LEFT JOIN interest_map map ON metrics.interest_id = map.id

    UNION

    SELECT *
    FROM interest_metrics metrics
    RIGHT JOIN interest_map map ON metrics.interest_id = map.id )

    SELECT
        COUNT(DISTINCT id) AS total_map_id,
        COUNT(DISTINCT interest_id) AS total_metricas_id,
        SUM(CASE WHEN id is NULL THEN 1 END) AS no_en_metrica,
        SUM(CASE WHEN interest_id is NULL THEN 1 END) AS no_en_metrica
    FROM
        full_outer_join_table

'''
pd.read_sql_query(query, con=engine)


2025-05-06 13:36:09,724 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:36:09,725 INFO sqlalchemy.engine.Engine 

    WITH full_outer_join_table AS (
   SELECT *
    FROM interest_metrics metrics
    LEFT JOIN interest_map map ON metrics.interest_id = map.id

    UNION

    SELECT *
    FROM interest_metrics metrics
    RIGHT JOIN interest_map map ON metrics.interest_id = map.id )

    SELECT
        COUNT(DISTINCT id) AS total_map_id,
        COUNT(DISTINCT interest_id) AS total_metricas_id,
        SUM(CASE WHEN id is NULL THEN 1 END) AS no_en_metrica,
        SUM(CASE WHEN interest_id is NULL THEN 1 END) AS no_en_metrica
    FROM
        full_outer_join_table


2025-05-06 13:36:09,725 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:36:10,195 INFO sqlalchemy.engine.Engine ROLLBACK


,total_map_id,total_metricas_id,no_en_metrica,no_en_metrica
0,1209,1202,None,7.0


# **Resume los valores de id en fresh_segments.interest_map por su recuento total de registros en esta tabla**


In [111]:
query = '''

    SELECT 
        COUNT(*) AS total_registros
    FROM 
        interest_map

'''
pd.read_sql_query(query, con=engine)


2025-05-06 13:37:56,524 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:37:56,524 INFO sqlalchemy.engine.Engine 

    SELECT 
        COUNT(*) AS total_registros
    FROM 
        interest_map


2025-05-06 13:37:56,525 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:37:56,529 INFO sqlalchemy.engine.Engine ROLLBACK


,total_registros
0,1209


# **¿Qué tipo de JOIN de tabla deberíamos realizar para nuestro análisis y por qué? Verifica tu lógica revisando las filas donde interest_id = 21246 en tu salida unida e incluye todas las columnas de fresh_segments.interest_metrics y todas las columnas de fresh_segments.interest_map excepto la columna id.**


In [113]:
query = '''

    SELECT *
    FROM 
        interest_map map
    INNER JOIN 
        interest_metrics metrics
    ON 
        map.id = metrics.interest_id
    WHERE 
        metrics.interest_id = 21246   
    AND 
        metrics._month IS NOT NULL;

'''
pd.read_sql_query(query, con=engine)


2025-05-06 13:40:43,434 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:40:43,435 INFO sqlalchemy.engine.Engine 

    SELECT *
    FROM 
        interest_map map
    INNER JOIN 
        interest_metrics metrics
    ON 
        map.id = metrics.interest_id
    WHERE 
        metrics.interest_id = 21246   
    AND 
        metrics._month IS NOT NULL;


2025-05-06 13:40:43,435 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:40:43,450 INFO sqlalchemy.engine.Engine ROLLBACK


,id,interest_name,interest_summary,created_at,last_modified,id_,_month,_year,month_year,interest_id,composition,index_value,ranking,percentile_ranking
0,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,727,7,2018,01-07-2018,21246,2.26,0.65,722,0.96
1,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,1499,8,2018,01-08-2018,21246,2.13,0.59,765,0.26
2,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,2279,9,2018,01-09-2018,21246,2.06,0.61,774,0.77
3,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,3140,10,2018,01-10-2018,21246,1.74,0.58,855,0.23
4,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,4050,11,2018,01-11-2018,21246,2.25,0.78,908,2.16
5,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,5056,12,2018,01-12-2018,21246,1.97,0.70,983,1.21
6,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,6020,1,2019,01-01-2019,21246,2.05,0.76,954,1.95
7,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,7150,2,2019,01-02-2019,21246,1.84,0.68,1109,1.07
8,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,8285,3,2019,01-03-2019,21246,1.75,0.67,1123,1.14
9,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,9392,4,2019,01-04-2019,21246,1.58,0.63,1092,0.64


# **¿Hay algún registro en tu tabla unida donde el valor de month_year sea anterior al valor de created_at de la tabla fresh_segments.interest_map? ¿Crees que estos valores son válidos y por qué?**






In [148]:
query = '''

    WITH resume_formated_table AS (
    SELECT 
        STR_TO_DATE(month_year , "%%d-%%m-%%Y") AS month_year_formated,
        STR_TO_DATE(created_at , "%%Y-%%m-%%d") as created_at_formated,
        id,
        interest_name,
        interest_summary,
        created_at,
        last_modified,
        id_, 
        _month, 
        _year,
        month_year,
        interest_id,
        composition,
        index_value,
        ranking,
        percentile_ranking
    FROM interest_map map
    INNER JOIN interest_metrics metrics
    ON map.id = metrics.interest_id )

    SELECT
        *
    FROM
        resume_formated_table
    WHERE
        month_year_formated < created_at_formated
         

'''
pd.read_sql_query(query, con=engine)


2025-05-06 13:57:21,748 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 13:57:21,749 INFO sqlalchemy.engine.Engine 

    WITH resume_formated_table AS (
    SELECT 
        STR_TO_DATE(month_year , "%%d-%%m-%%Y") AS month_year_formated,
        STR_TO_DATE(created_at , "%%Y-%%m-%%d") as created_at_formated,
        id,
        interest_name,
        interest_summary,
        created_at,
        last_modified,
        id_, 
        _month, 
        _year,
        month_year,
        interest_id,
        composition,
        index_value,
        ranking,
        percentile_ranking
    FROM interest_map map
    INNER JOIN interest_metrics metrics
    ON map.id = metrics.interest_id )

    SELECT
        *
    FROM
        resume_formated_table
    WHERE
        month_year_formated < created_at_formated
         


2025-05-06 13:57:21,750 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 13:57:21,801 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year_formated,created_at_formated,id,interest_name,interest_summary,created_at,last_modified,id_,_month,_year,month_year,interest_id,composition,index_value,ranking,percentile_ranking
0,2018-07-01,2018-07-06,32704,Major Airline Customers,People visiting sites for major airline brands...,2018-07-06 14:35:04,2018-07-06 14:35:04,228,7,2018,01-07-2018,32704,8.04,2.27,225,69.14
1,2018-07-01,2018-07-17,33191,Online Shoppers,People who spend money online,2018-07-17 10:40:03,2018-07-17 10:46:58,285,7,2018,01-07-2018,33191,3.99,2.11,283,61.18
2,2018-07-01,2018-07-06,32703,School Supply Shoppers,Consumers shopping for classroom supplies for ...,2018-07-06 14:35:04,2018-07-06 14:35:04,377,7,2018,01-07-2018,32703,5.53,1.80,375,48.56
3,2018-07-01,2018-07-06,32701,Womens Equality Advocates,People visiting sites advocating for womens eq...,2018-07-06 14:35:03,2018-07-06 14:35:03,486,7,2018,01-07-2018,32701,4.23,1.41,483,33.74
4,2018-07-01,2018-07-06,32705,Certified Events Professionals,Professionals reading industry news and resear...,2018-07-06 14:35:04,2018-07-06 14:35:04,509,7,2018,01-07-2018,32705,4.38,1.34,505,30.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2019-04-01,2019-04-15,49972,Horseback Riding Enthusiasts,People reading horseback riding news and resou...,2019-04-15 18:00:00,2019-04-24 17:40:04,9034,4,2019,01-04-2019,49972,2.11,1.15,722,34.30
184,2019-04-01,2019-04-08,49502,Veterinarians,Veterinarians,2019-04-08 18:00:05,2019-07-09 13:57:13,9069,4,2019,01-04-2019,49502,1.87,1.12,768,30.12
185,2019-04-01,2019-04-15,49974,Agricultural and Food Issues Researchers,People researching organizations for food and ...,2019-04-15 18:00:00,2019-04-24 17:40:04,9107,4,2019,01-04-2019,49974,2.00,1.10,799,27.30
186,2019-04-01,2019-04-15,49973,Farm Finance Researchers,People researching financial institutions spec...,2019-04-15 18:00:00,2019-04-24 17:40:04,9216,4,2019,01-04-2019,49973,1.66,1.03,910,17.20


In [153]:
query = '''

    WITH resume_formated_table AS (
    SELECT 
        STR_TO_DATE(month_year , "%%d-%%m-%%Y") AS month_year_formated,
        STR_TO_DATE(created_at , "%%Y-%%m-%%d") as created_at_formated,
        id,
        interest_name,
        interest_summary,
        created_at,
        last_modified,
        id_, 
        _month, 
        _year,
        month_year,
        interest_id,
        composition,
        index_value,
        ranking,
        percentile_ranking
    FROM interest_map map
    INNER JOIN interest_metrics metrics
    ON map.id = metrics.interest_id )

    SELECT
        *
    FROM
        resume_formated_table
    WHERE
        month_year_formated < DATE_FORMAT(created_at_formated, '%%m' )
         

'''
pd.read_sql_query(query, con=engine)


2025-05-06 14:01:57,760 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 14:01:57,761 INFO sqlalchemy.engine.Engine 

    WITH resume_formated_table AS (
    SELECT 
        STR_TO_DATE(month_year , "%%d-%%m-%%Y") AS month_year_formated,
        STR_TO_DATE(created_at , "%%Y-%%m-%%d") as created_at_formated,
        id,
        interest_name,
        interest_summary,
        created_at,
        last_modified,
        id_, 
        _month, 
        _year,
        month_year,
        interest_id,
        composition,
        index_value,
        ranking,
        percentile_ranking
    FROM interest_map map
    INNER JOIN interest_metrics metrics
    ON map.id = metrics.interest_id )

    SELECT
        *
    FROM
        resume_formated_table
    WHERE
        month_year_formated < DATE_FORMAT(created_at_formated, '%%m' )
         


2025-05-06 14:01:57,761 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-06 14:01:57,816 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year_formated,created_at_formated,id,interest_name,interest_summary,created_at,last_modified,id_,_month,_year,month_year,interest_id,composition,index_value,ranking,percentile_ranking


Correspomden a valores validos ya que para todas las fechas menores se cumple que se encuentran en el mismo mes.

In [ ]:
)